In [1]:
import sys
print(sys.executable)

/home/nick/envs/grag-llama/bin/python


In [ ]:
from GlirelPathExtractor import GlirelPathExtractor # the import is a bit strage??
from llama_index.core import SimpleDirectoryReader, PropertyGraphIndex
from llama_index.core.graph_stores import SimpleGraphStore
from llama_index.core import Settings
from IPython.display import Markdown, display


/home/nick/envs/grag-llama/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from llama_index.llms.ollama import Ollama

In [5]:
llm = Ollama(
    model= "gemma3:12b",
    request_timeout=120.0,
    context_window=8128,
    temperature=0.0
)

Settings.llm = llm
Settings.chunk_size=512
Settings.chunk_overlap=64

In [6]:
from llama_index.embeddings.ollama import OllamaEmbedding
embed_model = OllamaEmbedding(
    model_name="snowflake-arctic-embed2:latest",
    ollama_additional_kwargs={"mirostat": 0},
)
Settings.embed_model = embed_model

In [7]:
documents = SimpleDirectoryReader(
    input_files=["../.data/Demo-Barack_Obama.txt"]
).load_data()

In [8]:
extractor = GlirelPathExtractor(device="cuda")


In [9]:
index = PropertyGraphIndex.from_documents(
    documents,
    kg_extractors=[extractor],
    use_async = False
    
)


Fetching 5 files: 100%|██████████| 5/5 [00:00<00:00, 86659.17it/s]
/home/nick/envs/grag-llama/lib/python3.12/site-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [ ]:
query_engine = index.as_query_engine(
    llm=llm,
    include_text="True",
    response_mode="tree_summarize",
    embedding_mode="hybrid",
    similarity_top_k=5,
    )

In [ ]:
response= query_engine.query("When was obama born?")
response